## CSC 575 HW#4

https://nbviewer.jupyter.org/url/condor.depaul.edu/ntomuro/courses/575/assign/575hw4.ipynb

<p><strong>Overview</strong>:</p>
<p>Implement the 'Inverted Index Retrieval Algorithm' (in
<a href="http://condor.depaul.edu/ntomuro/courses/575/notes/VS-Retrieval.pptx">
lecture note (#6)</a>) and the evaluation metric Mean Average Precision (MAP) 
(in <a href="http://condor.depaul.edu/ntomuro/courses/575/notes/Evaluation.pptx">
lecture note (#7)</a>), and apply to a corpus called <a href="http://ir.dcs.gla.ac.uk/resources/test_collections/">Medline 
collection</a>.&nbsp; 
</p>
<p>The Medline 
collection is one of the Information Retrieval (IR) standard test 
collections, which have been used by many researchers as benchmark to evaluate IR 
systems.&nbsp; It contains 1033 documents (abstracts of papers published on 
Medline), 30 queries and relevance judgments of all query-document pairs.&nbsp; 
</p>

### Programming: Vector-space Retrieval & Evaluation -- Partially filled code

### (1) Step 1: Load Inverted Index (H) and compute DocLen (DL).

In [3]:
import csv
import math

tindexfile = 'medline_term_index.csv'
invindexfile = 'medline_inverted_index.csv'
dindexfile = 'medline_doc_index.csv'

# Number of documents in the corpus (hard-coded for this corpus)
N = 1033

# Major data structures
H_invindex = {} # inverted index; term -> (idf, L:hashmap of (docID . tf))
DL_doclen = {}  # document lengths; docID -> len

## (1) Read the term index file and populate the invindex first
tid2term_map = {} # temporary storage to hold mappings of termID -> term

fin = open(tindexfile, 'r', encoding='utf-8')
reader = csv.reader(fin, delimiter='\t')
for line in reader:
    term = line[0]    # term string
    termID = line[1]  # termID
    df = int(line[2]) # document frequency
    idf = math.log10(N/df) # idf
    # record term -> (idf, emptyL) in H
    H_invindex[term] = (idf, dict())
    # record termID -> term 
    tid2term_map[termID] = term 
fin.close()

## (2) Read the inverted index file and add postings lists in H.
## Also compute document lengths too, incrementally -- and record in DL.
fin = open(invindexfile, 'r')
reader = csv.reader(fin, delimiter='\t')
for line in reader:
    termID = line[0]
    idx = 1
    while idx < (len(line)-1):
        docID = line[idx]
        tf = int(line[idx+1]) # raw tf of the term in this document
        # Record docID -> tf in term's L
        L = (H_invindex[tid2term_map[termID]])[1]
        L[docID] = tf  # docID -> raw term frequency
        
        # Accumulate the component vector length for the document
        tfidf = tf * (H_invindex[tid2term_map[termID]])[0] # tf * idf
        tfidfsq = math.pow(tfidf, 2.0)
        if docID in DL_doclen:
            DL_doclen[docID] += tfidfsq
        else:
            DL_doclen[docID] = tfidfsq
        #
        idx += 2
fin.close()

# Fix the DL entries by applying sqrt to make vector length.
for docID in DL_doclen.keys():
    val = DL_doclen[docID]
    DL_doclen[docID] = math.sqrt(val)

    
print ('Total # terms: %d' % len(H_invindex))
for term in ['pentobarbit', 'defici', 'treatment']:
    print (' - Entry for \'%s\': df=%s, idf=%s' % (term, len(H_invindex[term][1]), H_invindex[term][0]))

print ('\nTotal # documents: %d' % len(DL_doclen))
for docID in ['59', '1033']:
    print (' - Vector len for Doc %s = %s' % (docID, DL_doclen[docID]))


Total # terms: 11463
 - Entry for 'pentobarbit': df=4, idf=2.412040330191658
 - Entry for 'defici': df=39, idf=1.4230357144931214
 - Entry for 'treatment': df=172, idf=0.7785718746120717

Total # documents: 1033
 - Vector len for Doc 59 = 13.811725366348801
 - Vector len for Doc 1033 = 31.163653356034512


### (2) Step 2: Queries as Vectors

In [4]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

queryfile = 'medline.query'

# A list of queries. Each Query is a tuple, (qID, Q:term->tf map)
Queries_list = []

fin = open(queryfile, 'r', encoding='utf-8')#'iso-8859-1')
porter = nltk.PorterStemmer()

for line in fin:
    matchObj = re.match(r'^(\d+)\s+(.*)', line)
    if not matchObj:
        print ("ERROR with line -- %s" % line)
    else:
        queryID = matchObj.group(1) # queryID
        text = matchObj.group(2)    # query string (ignoring sentences)

        # process text string -- same processing as one applied to documents.
        tokens = word_tokenize(text.lower())
        terms = [porter.stem(w) for w in tokens if w not in stopwords.words('english') and len(w) > 1] # (**)
        # term frequencies of the terms in this query are obtained by NLTK's FreqDist
        fdist = nltk.FreqDist(terms)
        # append the Query in the list
        Queries_list.append((queryID, dict(fdist)))
fin.close()

print ('Total # queries: %d' % len(Queries_list))
for qid in [1, 21]:
    print (' - Query %s: %s' % (Queries_list[qid][0], Queries_list[qid][1]))

Total # queries: 30
 - Query 2: {'relationship': 1, 'blood': 1, 'cerebrospin': 1, 'fluid': 1, 'oxygen': 1, 'concentr': 1, 'partial': 1, 'pressur': 1, 'method': 1, 'interest': 1, 'polarographi': 1}
 - Query 22: {'mycoplasma': 1, 'infect': 1, 'presenc': 1, 'embryo': 1, 'fetu': 1, 'newborn': 1, 'infant': 1, 'anim': 1, 'pregnanc': 1, 'gynecolog': 1, 'diseas': 1, 'relat': 1, 'chromosom': 2, 'abnorm': 1}


In [5]:
print ('Total # terms: %d' % len(H_invindex))
for term in ['pentobarbit']:
    print (' - Entry for \'%s\': df=%s, idf=%s' % (term, len(H_invindex[term][1]), H_invindex[term][0]))
    print(H_invindex[term])
    print()

print ('\nTotal # documents: %d' % len(DL_doclen))
for docID in ['59', '1033']:
    print (' - Vector len for Doc %s = %s' % (docID, DL_doclen[docID]))

Total # terms: 11463
 - Entry for 'pentobarbit': df=4, idf=2.412040330191658
(2.412040330191658, {'187': 1, '301': 1, '416': 1, '419': 1})


Total # documents: 1033
 - Vector len for Doc 59 = 13.811725366348801
 - Vector len for Doc 1033 = 31.163653356034512


## (3) Retrieval and Ranking -- Step 3,4,5 for each Query

### - For each query, follow Step 3,4,5 of the Vector Space Retrieval Algorithm and obtain a ranked list of retrieved documents (sorted by the cosine measure) -- 'R' in the algorithm.  
### - Then save the ranked lists in a list (in the same order of the query) -- to be used in the next step/Evaluation.
### - (**) Also, WRITE the ranked lists to an output file.  See the homework page for details.

In [6]:
i = 0
for key in H_invindex:
    print(key+' '+str(H_invindex[key]))
    if(i==5):break
    i+=1



'' (1.4230357144931214, {'497': 1, '545': 1, '556': 1, '560': 1, '565': 1, '592': 2, '602': 2, '604': 4, '606': 1, '608': 6, '611': 1, '612': 2, '613': 1, '615': 2, '624': 1, '625': 1, '630': 2, '633': 2, '640': 1, '647': 1, '660': 1, '678': 1, '691': 1, '699': 1, '709': 1, '713': 1, '715': 2, '720': 2, '738': 1, '740': 3, '743': 1, '750': 1, '752': 1, '789': 1, '798': 1, '801': 1, '807': 1, '808': 3, '817': 2})
'a (2.7130703258556395, {'421': 1, '609': 1})
'achondroplast (3.0141003215196207, {'576': 1})
'adequ (3.0141003215196207, {'421': 1})
'agnos (3.0141003215196207, {'358': 2})
'air (3.0141003215196207, {'70': 1})


In [122]:
# RETRIEVE IDF IN QUERY
from collections import defaultdict
from scipy.spatial import distance
import numpy as np
import pandas as pd
docs_tfidf = defaultdict(list)
i=0
f=open('rankedlist.txt','w')

############### For each token, T, in Q: ###############
for q in Queries_list:
#     doc_w_terms=defaultdict(list)
#     doc_w_terms=[]
    q_tfidf=[]
    doc_score={}
########################################################################################################################
### Let Ibe the IDF of T, and Kbe the frequency of T in Q;
### Set the weight of T in Q: W= K*I; (tf*idffor T in Q)
### If a term in a query was not in the vocabulary (the dictionary structure made from the documents), ignore/skip it.
    for token in (q[1].keys()): # ITERATE OVER QUERY TERMS
        if(token not in H_invindex.keys()):
            continue
#         print("QUERY TERM: "+token+' '+str(H_invindex[token][0])+'x'+str(q[1][token]))
        qt_tfidf=H_invindex[token][0]*q[1][token]
        q_tfidf.append(qt_tfidf)
#         print("DOCS: ",end="")
#         print(H_invindex[token][1])
########################################################################################################################
###     Let L be the posting list of T from H;
###     For each HashMap, O, in L:
        for docid in H_invindex[token][1].keys():
#             print('\t'+str(docid)+': '+str(H_invindex[token][1][docid])+' x '+str(H_invindex[token][0])+' = '
#                   +str(H_invindex[token][1][docid]*H_invindex[token][0]))
########################################################################################################################
###         Increment D’s score by W* C * I; (tf*idffor T in Q x tf*idffor T in D)
            if (docid not in doc_score.keys()):
                doc_score[docid]=0
            doc_score[docid]+=qt_tfidf*H_invindex[token][1][docid]*H_invindex[token][0]
########################################################################################################################
### Compute the length, L, of the vector Q (square-root of the sum of the squares of its weights).
    q_len=sum(np.array(q_tfidf)**2)
### For each retrieved document D in R:
###     Let S be the current accumulated score of D;
###     (S is the dot-product of D and Q)
###     Let Y be the length of D in DL;
###     Let D’s final score to be S/(L * Y); (the cosine)
    for docid in doc_score.keys():
        doc_score[docid] = doc_score[docid] / (q_len*DL_doclen[docID])
    if(i<3):
        print(q[0], end=" ")
    f.write(q[0]+' ')
    for w in sorted(doc_score, key=doc_score.get, reverse=True):
#         if(i<3):
#             print(str(w) + '('+str(doc_score[w])+')', end=" ")
        f.write(w+' ')
    f.write('\n')
    if(i<3):print('\n')

    i+=1
#     if(i==3):break
f.close()

1 

2 

3 



In [11]:
q = np.array(q_tfidf)
testdoc = np.array(query_docs[0:1][:])[0]
print(Queries_list[0])
print(q)
print(testdoc)
distance.cosine(testdoc,q)
# print(distance.cosine(np.array(query_docs[0:1][:])[0],np.array(q_tdidf)))
docs_tfidf
####################################
# PROBLEM WAS WRONG ORDER OF KEY IN DEFAULT DICT docs_tfidf

('1', {'crystallin': 1, 'len': 1, 'vertebr': 1, 'includ': 1, 'human': 1})
[2.16900228 0.95719547 1.13903906 1.40131646 2.41204033]
[8.67600913 4.20394939 0.         0.         0.        ]


defaultdict(list,
            {'crystallin': [{'72': 8.676009126021455},
              {'175': 2.169002281505364},
              {'181': 2.169002281505364},
              {'336': 2.169002281505364},
              {'500': 10.845011407526819},
              {'509': 2.169002281505364},
              {'549': 2.169002281505364}],
             'human': [{'9': 1.914390940366296},
              {'11': 2.871586410549444},
              {'41': 2.871586410549444},
              {'42': 0.957195470183148},
              {'58': 4.78597735091574},
              {'65': 4.78597735091574},
              {'67': 0.957195470183148},
              {'70': 0.957195470183148},
              {'78': 1.914390940366296},
              {'82': 0.957195470183148},
              {'83': 0.957195470183148},
              {'87': 0.957195470183148},
              {'95': 0.957195470183148},
              {'99': 3.828781880732592},
              {'108': 0.957195470183148},
              {'125': 0.957195470183148},
         

## (4) Evaluation -- Compute MAP
### - Read the relevancy answers from the file "medline.rel".
### - Compare the ranked lists with the anwers, and compute the MAP score.
### - (**) Also print the MAP score (to the terminal).

In [167]:
import requests
r=requests.get("http://condor.depaul.edu/ntomuro/courses/575/assign/medline.rel")
r.content
rel_docs={}
for line in r.text.split('\n'):
#     print(line)
    line=(line.strip().split(' '))
    if(line[0]!=''):
        rel_docs[line[0]]=line[1:]
#     print(rel_docs[line[0]])
print(rel_docs)

{'1': ['13', '14', '15', '72', '79', '138', '142', '164', '165', '166', '167', '168', '169', '170', '171', '172', '180', '181', '182', '183', '184', '185', '186', '211', '212', '499', '500', '501', '502', '503', '504', '506', '507', '508', '510', '511', '513'], '2': ['80', '90', '162', '187', '236', '237', '258', '289', '290', '292', '293', '294', '296', '300', '301', '303'], '3': ['59', '62', '67', '69', '70', '71', '73', '78', '81', '160', '163', '230', '231', '232', '233', '234', '276', '277', '279', '282', '283', '287'], '4': ['93', '94', '96', '141', '173', '174', '175', '176', '177', '178', '207', '208', '209', '210', '259', '396', '397', '399', '400', '404', '405', '406', '408'], '5': ['1', '2', '4', '5', '6', '7', '8', '9', '10', '11', '12', '158', '159', '188', '304', '305', '306', '307', '325', '326', '327', '329', '330', '331', '332', '333'], '6': ['112', '115', '116', '118', '122', '238', '239', '242', '260', '309', '320', '321', '323'], '7': ['92', '121', '189', '247', '26

• Consider rank position of each relevantdoc  
    –K1, K2, … KR  
• Compute Precision@Kfor each K1, K2, … KR  
• Average precision = average of P@K  
• Ex: has AvgPrecof  
• MAP is the average of the average precision value for a set of queries.

In [126]:
retrived_docs={}
f=open('rankedlist.txt','r')
lines=f.readlines()
for line in lines:
#     print(line)
    line=line.strip().split(' ')
    retrived_docs[line[0]]=line[1:]

In [162]:
print(rel_docs['1'])
print()
print(retrived_docs['1'])

['13', '14', '15', '72', '79', '138', '142', '164', '165', '166', '167', '168', '169', '170', '171', '172', '180', '181', '182', '183', '184', '185', '186', '211', '212', '499', '500', '501', '502', '503', '504', '506', '507', '508', '510', '511', '513']

['500', '965', '72', '15', '212', '166', '513', '499', '511', '360', '181', '171', '182', '637', '168', '509', '206', '142', '172', '512', '401', '13', '14', '79', '164', '165', '167', '169', '170', '183', '184', '727', '138', '180', '175', '336', '549', '58', '65', '570', '41', '403', '542', '185', '186', '211', '504', '506', '510', '99', '177', '640', '758', '838', '259', '645', '11', '173', '174', '209', '256', '540', '541', '642', '700', '763', '876', '899', '986', '112', '913', '125', '127', '231', '469', '213', '501', '502', '503', '505', '507', '508', '9', '78', '130', '158', '163', '178', '215', '333', '404', '445', '567', '578', '584', '589', '590', '593', '603', '652', '654', '658', '660', '730', '734', '816', '869', '870', 

In [175]:
for qid in rel_docs.keys():
    print('QUERY: '+qid+' : '+str(len(rel_docs[qid]))+' rel docs')
#     print(rel_docs[qid])
    print('RETRIEVED: '+str(len(retrived_docs[qid]))+' docs')
    for i in range(1,len(retrived_docs[qid])):
        retrived=set(retrived_docs[qid][:i])
#         print(retrived.intersection(rel_docs[qid]))
        print(len(retrived.intersection(rel_docs[qid]))/len(retrived))
#         print(retrived_docs[qid][:i],end='\n')
#     for doc in retrived_docs[qid]:
#         print(doc,end=" ")
    print('\n')
    

QUERY: 1 : 37 rel docs
RETRIEVED: 223 docs
1.0
0.5
0.6666666666666666
0.75
0.8
0.8333333333333334
0.8571428571428571
0.875
0.8888888888888888
0.8
0.8181818181818182
0.8333333333333334
0.8461538461538461
0.7857142857142857
0.8
0.75
0.7058823529411765
0.7222222222222222
0.7368421052631579
0.7
0.6666666666666666
0.6818181818181818
0.6956521739130435
0.7083333333333334
0.72
0.7307692307692307
0.7407407407407407
0.75
0.7586206896551724
0.7666666666666667
0.7741935483870968
0.75
0.7575757575757576
0.7647058823529411
0.7428571428571429
0.7222222222222222
0.7027027027027027
0.6842105263157895
0.6666666666666666
0.65
0.6341463414634146
0.6190476190476191
0.6046511627906976
0.6136363636363636
0.6222222222222222
0.6304347826086957
0.6382978723404256
0.6458333333333334
0.6530612244897959
0.64
0.6274509803921569
0.6153846153846154
0.6037735849056604
0.5925925925925926
0.5818181818181818
0.5714285714285714
0.5614035087719298
0.5517241379310345
0.5423728813559322
0.5333333333333333
0.5245901639344263

0.07027027027027027
0.06989247311827956
0.06951871657754011
0.06914893617021277
0.06878306878306878
0.06842105263157895
0.06806282722513089
0.06770833333333333
0.06735751295336788
0.06701030927835051
0.06666666666666667
0.0663265306122449
0.06598984771573604
0.06565656565656566
0.06532663316582915
0.065
0.06467661691542288
0.06435643564356436
0.06403940886699508
0.06372549019607843
0.06341463414634146
0.06310679611650485
0.06280193236714976
0.0625
0.06220095693779904
0.06190476190476191
0.061611374407582936
0.06132075471698113
0.06103286384976526
0.06074766355140187
0.06046511627906977
0.06018518518518518
0.059907834101382486
0.05963302752293578
0.0593607305936073
0.05909090909090909
0.058823529411764705
0.05855855855855856
0.05829596412556054
0.05803571428571429
0.057777777777777775
0.05752212389380531
0.05726872246696035
0.05701754385964912
0.056768558951965066
0.05652173913043478
0.05627705627705628
0.05603448275862069
0.055793991416309016
0.05555555555555555
0.05531914893617021
0.0

0.018456375838926176
0.018425460636515914
0.01839464882943144
0.018363939899833055
0.018333333333333333
0.018302828618968387
0.018272425249169437
0.01824212271973466
0.018211920529801324
0.01818181818181818
0.018151815181518153
0.018121911037891267
0.018092105263157895
0.0180623973727422
0.018032786885245903
0.01800327332242226
0.017973856209150325
0.01794453507340946
0.017915309446254073
0.01788617886178862
0.017857142857142856
0.017828200972447326
0.01779935275080906
0.017770597738287562
0.017741935483870968
0.017713365539452495
0.017684887459807074
0.01765650080256822
0.017628205128205128
0.0176
0.01757188498402556
0.017543859649122806
0.01751592356687898
0.017488076311605722
0.01746031746031746
0.017432646592709985
0.01740506329113924
0.017377567140600316
0.017350157728706624
0.01732283464566929
0.01729559748427673
0.01726844583987441
0.017241379310344827


QUERY: 9 : 28 rel docs
RETRIEVED: 434 docs
0.0
0.5
0.6666666666666666
0.5
0.6
0.6666666666666666
0.5714285714285714
0.5
0.5555

0.02962962962962963
0.02952029520295203
0.029411764705882353
0.029304029304029304
0.029197080291970802
0.02909090909090909
0.028985507246376812
0.02888086642599278
0.02877697841726619
0.02867383512544803
0.02857142857142857
0.028469750889679714
0.028368794326241134
0.028268551236749116
0.028169014084507043
0.028070175438596492
0.027972027972027972
0.027874564459930314
0.027777777777777776
0.02768166089965398
0.027586206896551724
0.027491408934707903
0.0273972602739726
0.027303754266211604
0.027210884353741496
0.02711864406779661
0.02702702702702703
0.026936026936026935
0.026845637583892617
0.026755852842809364
0.02666666666666667
0.026578073089700997
0.026490066225165563
0.026402640264026403
0.02631578947368421
0.02622950819672131
0.026143790849673203
0.026058631921824105
0.025974025974025976
0.025889967637540454
0.025806451612903226
0.02572347266881029
0.02564102564102564
0.025559105431309903
0.025477707006369428
0.025396825396825397
0.02531645569620253
0.025236593059936908
0.02515723

0.06220095693779904
0.06190476190476191
0.061611374407582936
0.06132075471698113
0.06103286384976526
0.06074766355140187
0.06046511627906977
0.06018518518518518
0.059907834101382486
0.05963302752293578
0.0593607305936073
0.05909090909090909
0.058823529411764705
0.05855855855855856
0.05829596412556054
0.05803571428571429
0.057777777777777775
0.05752212389380531
0.05726872246696035
0.05701754385964912
0.056768558951965066
0.05652173913043478
0.05627705627705628
0.05603448275862069
0.055793991416309016
0.05555555555555555
0.05531914893617021
0.05508474576271186
0.05485232067510549
0.0546218487394958
0.05439330543933055
0.05416666666666667
0.05394190871369295
0.05371900826446281
0.053497942386831275
0.05327868852459016
0.053061224489795916
0.052845528455284556
0.05263157894736842
0.05241935483870968
0.05220883534136546
0.052
0.05179282868525897
0.051587301587301584
0.05138339920948617
0.051181102362204724
0.050980392156862744
0.05078125
0.05058365758754864
0.050387596899224806
0.0501930501

0.2549019607843137
0.25
0.24528301886792453
0.24074074074074073
0.2545454545454545
0.25
0.24561403508771928
0.2413793103448276
0.23728813559322035
0.23333333333333334
0.22950819672131148
0.24193548387096775
0.23809523809523808
0.234375
0.23076923076923078
0.22727272727272727
0.22388059701492538
0.23529411764705882
0.2318840579710145
0.22857142857142856
0.22535211267605634
0.2361111111111111
0.2328767123287671
0.22972972972972974
0.24
0.23684210526315788
0.23376623376623376
0.23076923076923078
0.22784810126582278
0.2375
0.2345679012345679
0.23170731707317074
0.2289156626506024
0.2261904761904762
0.2235294117647059
0.22093023255813954
0.21839080459770116
0.2159090909090909
0.21348314606741572
0.2222222222222222
0.21978021978021978
0.21739130434782608
0.21505376344086022
0.2127659574468085
0.21052631578947367
0.20833333333333334
0.21649484536082475
0.21428571428571427
0.21212121212121213
0.21
0.2079207920792079
0.20588235294117646
0.20388349514563106
0.20192307692307693
0.2095238095238095

0.8333333333333334
0.8421052631578947
0.85
0.8571428571428571
0.8181818181818182
0.782608695652174
0.75
0.72
0.6923076923076923
0.6666666666666666
0.6785714285714286
0.6551724137931034


QUERY: 24 : 22 rel docs
RETRIEVED: 681 docs
0.0
0.5
0.6666666666666666
0.75
0.8
0.8333333333333334
0.8571428571428571
0.75
0.7777777777777778
0.8
0.8181818181818182
0.75
0.7692307692307693
0.7857142857142857
0.8
0.8125
0.8235294117647058
0.7777777777777778
0.7368421052631579
0.75
0.7142857142857143
0.7272727272727273
0.6956521739130435
0.6666666666666666
0.68
0.6538461538461539
0.6296296296296297
0.6071428571428571
0.5862068965517241
0.5666666666666667
0.5483870967741935
0.5625
0.5454545454545454
0.5588235294117647
0.5428571428571428
0.5555555555555556
0.5405405405405406
0.5263157894736842
0.5128205128205128
0.5
0.5121951219512195
0.5
0.4883720930232558
0.4772727272727273
0.4666666666666667
0.45652173913043476
0.46808510638297873
0.4583333333333333
0.4489795918367347
0.44
0.43137254901960786
0.42307692

0.10810810810810811
0.10762331838565023
0.10714285714285714
0.10666666666666667
0.10619469026548672
0.10572687224669604
0.10526315789473684
0.10480349344978165
0.10434782608695652
0.1038961038961039
0.10344827586206896
0.10300429184549356
0.10256410256410256
0.10212765957446808
0.1016949152542373
0.10126582278481013
0.10084033613445378
0.100418410041841
0.1
0.0995850622406639
0.09917355371900827
0.09876543209876543
0.09836065573770492
0.09795918367346938
0.0975609756097561
0.09716599190283401
0.0967741935483871
0.0963855421686747
0.096
0.09561752988047809
0.09523809523809523
0.09486166007905138
0.09448818897637795
0.09411764705882353
0.09375
0.0933852140077821
0.09302325581395349
0.09266409266409266
0.09230769230769231
0.09195402298850575
0.0916030534351145
0.09125475285171103
0.09090909090909091
0.09056603773584905
0.09022556390977443
0.0898876404494382
0.08955223880597014
0.08921933085501858
0.08888888888888889
0.08856088560885608
0.08823529411764706
0.08791208791208792
0.08759124087

0.3069306930693069
0.30392156862745096
0.30097087378640774
0.2980769230769231
0.29523809523809524
0.29245283018867924
0.2897196261682243
0.28703703703703703
0.28440366972477066
0.2818181818181818
0.27927927927927926
0.2767857142857143
0.2743362831858407
0.2719298245614035
0.26956521739130435
0.2672413793103448
0.26495726495726496
0.2627118644067797
0.2605042016806723
0.25833333333333336
0.256198347107438
0.2540983606557377
0.25203252032520324
0.25
0.248
0.24603174603174602
0.2440944881889764
0.2421875
0.24031007751937986
0.23846153846153847
0.2366412213740458
0.23484848484848486
0.23308270676691728
0.23134328358208955
0.22962962962962963
0.22794117647058823
0.22627737226277372
0.2246376811594203
0.22302158273381295
0.22142857142857142
0.2198581560283688
0.21830985915492956
0.21678321678321677
0.2152777777777778
0.21379310344827587
0.21232876712328766
0.2108843537414966
0.20945945945945946
0.2080536912751678
0.20666666666666667
0.2052980132450331
0.21052631578947367
0.20915032679738563
